In [1]:
from datasets import load_dataset, load_metric
import datasets
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import random
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

2022-12-28 08:51:18.721184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-28 08:51:19.276344: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-28 08:51:19.276388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-28 08:51:19.276395: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [4]:
task = "mnli"
model_checkpoint = "symanto/mpnet-base-snli-mnli"


In [5]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Using the latest cached version of the module from /home/nico/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Mon Dec 12 10:10:25 2022) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
Found cached dataset glue (/home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_5365/1389288479.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', actual_task)
Using the latest cached version of the module from /home/nico/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Mon Dec 12 10:42:07 2022) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


In [6]:
#%pip install scikit-learn


In [7]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset["train"])

,premise,hypothesis,label,idx
0,"In midocean, the T-Rex wakes up and somehow breaks out of its heavily secured cargo hold, eats everybody on board, then cleverly scurries back into hiding.",The T-Rex knows how to pick locks on a secured cargo hold.,neutral,138700
1,I think I know of a man who may be in need of a change.,The man was too afraid to lead.,contradiction,228446
2,Poor Newt.,Lucky Newt.,contradiction,344034
3,"As Scandinavians and Bavarians cycle by, 1968 seems as remote as when the first cattle-herders came here from the other side of India some 3,500 years ago.","This part of India was uninhabited until 2,500 years ago.",contradiction,107337
4,The leptin breakthrough spurred efforts to manipulate the brain's hunger centers.,There have never been any efforts to manipulate the brain's hunger centers.,contradiction,331510
5,"Woodward did advance the story by reporting on the $50,000 donation for the university chair honoring Gore's sister and the DOE contract expansion.","The $50,000 donation was reported on by Woodward.",entailment,345480
6,Clean Coal Technology Regulatory Incentives,Dirty coal technology regulation incentives.,contradiction,255428
7,"The female palms make a less esoteric annual contribution of 10,000 tons of dates, which ripen in December and are prized for their juicy sweetness.","The male palms produce around 12,000 dates every year.",neutral,167651
8,The Church of Saint Barbara is also worth visiting; it is typical Coptic in style.,"It hasn't always been in a Coptic style, it was once styled in a typical Scandinavian sense.",neutral,41774
9,"By 2000, he had diversified funding sources for his then-$5.","By the year 2000, he had found some way to diversify his funding sources for just, what at the time, was $5, and this was unheard of by anyone before.",neutral,55046


In [8]:
#print(metric)

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
print(metric.compute(predictions=fake_preds, references=fake_labels,))


{'accuracy': 0.5}


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [10]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.", truncation=True)

{'input_ids': [0, 7596, 1014, 2027, 2032, 6255, 1003, 2, 2, 2002, 2027, 6255, 3636, 2011, 2013, 1016, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [12]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Conceptually cream skimming has two basic dimensions - product and geography.
Sentence 2: Product and geography are what make cream skimming work. 


In [13]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [14]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 17162, 2139, 6953, 8305, 25061, 2042, 2052, 3941, 9650, 1015, 4035, 2002, 10509, 1016, 2, 2, 4035, 2002, 10509, 2028, 2058, 2195, 6953, 8305, 25061, 2151, 1016, 2], [0, 2021, 2117, 2080, 2000, 2165, 2002, 1049, 3988, 2016, 2016, 2119, 2508, 7914, 2021, 4562, 2072, 2004, 2000, 2283, 2508, 2069, 2069, 2031, 5634, 2004, 9135, 2000, 2000, 6691, 2140, 2000, 13984, 5634, 2004, 2659, 2004, 9135, 1041, 3128, 2017, 6424, 1041, 2063, 1041, 3317, 1041, 3128, 3636, 2043, 2004, 5676, 2036, 2002, 1041, 2313, 1041, 3128, 3636, 2043, 2004, 5676, 2036, 2, 2, 2021, 4562, 2000, 2481, 2004, 2000, 2210, 2508, 2069, 2000, 2115, 9135, 1016, 2], [0, 2032, 2001, 2260, 2197, 2101, 4291, 2045, 2119, 8132, 3375, 2139, 1016, 2, 2, 1041, 2270, 2001, 2030, 2140, 2101, 15393, 2119, 4453, 2011, 14273, 11722, 1016, 2], [0, 2133, 2083, 2021, 2117, 1033, 2039, 2027, 2007, 2041, 2596, 2157, 1016, 2, 2, 2027, 2596, 7464, 2004, 2072, 1016, 2], [0, 3402, 1049, 2429, 2021, 2058, 2299, 2069, 2021, 2179, 3980

Preprocessing

In [15]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-fafca22d2ce8cb93.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f7baaf63ba62c6e7.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4241d8251a4fa825.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-2de4f90b732abfa1.arrow
Loading cached processed dataset at /home/nico/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-456f42c322561ed7.arrow


In [16]:

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [17]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]
batch_size = 4
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=0.05,
    weight_decay=0.5,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [18]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [20]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #optimizers=(transformers.AdamW, transformers.get_scheduler("linear", num_warmup_steps=0, num_training_steps=args.num_train_epochs*len(encoded_dataset["train"])/args.per_device_train_batch_size)),
    #token=token
)

/home/nico/Documents/IAssistant/my2/mpnet-base-snli-mnli-finetuned-mnli is already a clone of https://huggingface.co/NicolasLe/mpnet-base-snli-mnli-finetuned-mnli. Make sure you pull the latest changes with `repo.git_pull()`.


: 

In [20]:
trainer.train()



The following columns in the training set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
/home/nico/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 392702
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4909
  Number of trainable parameters = 109488771


  0%|          | 0/4909 [00:00<?, ?it/s]

You're using a MPNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.3064, 'learning_rate': 2.6944387859034425e-10, 'epoch': 0.01}
{'loss': 0.3065, 'learning_rate': 2.388877571806885e-10, 'epoch': 0.01}
{'loss': 0.2969, 'learning_rate': 2.083316357710328e-10, 'epoch': 0.02}
{'loss': 0.3051, 'learning_rate': 1.7777551436137705e-10, 'epoch': 0.02}
{'loss': 0.3051, 'learning_rate': 1.4721939295172133e-10, 'epoch': 0.03}
{'loss': 0.2953, 'learning_rate': 1.166632715420656e-10, 'epoch': 0.03}
{'loss': 0.3074, 'learning_rate': 8.610715013240986e-11, 'epoch': 0.04}
{'loss': 0.2797, 'learning_rate': 5.5551028722754124e-11, 'epoch': 0.04}
{'loss': 0.325, 'learning_rate': 2.499490731309839e-11, 'epoch': 0.05}


The following columns in the evaluation set don't have a corresponding argument in `MPNetForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx. If premise, hypothesis, idx are not expected by `MPNetForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 4


...

Saving model checkpoint to mpnet-base-snli-mnli-finetuned-mnli/checkpoint-4909
Configuration saved in mpnet-base-snli-mnli-finetuned-mnli/checkpoint-4909/config.json


{'eval_loss': 0.3395914137363434, 'eval_accuracy': 0.8757004584819155, 'eval_runtime': 44.683, 'eval_samples_per_second': 219.658, 'eval_steps_per_second': 54.92, 'epoch': 0.05}


Model weights saved in mpnet-base-snli-mnli-finetuned-mnli/checkpoint-4909/pytorch_model.bin
tokenizer config file saved in mpnet-base-snli-mnli-finetuned-mnli/checkpoint-4909/tokenizer_config.json
Special tokens file saved in mpnet-base-snli-mnli-finetuned-mnli/checkpoint-4909/special_tokens_map.json
tokenizer config file saved in mpnet-base-snli-mnli-finetuned-mnli/tokenizer_config.json
Special tokens file saved in mpnet-base-snli-mnli-finetuned-mnli/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from mpnet-base-snli-mnli-finetuned-mnli/checkpoint-4909 (score: 0.8757004584819155).


{'train_runtime': 535.8663, 'train_samples_per_second': 36.642, 'train_steps_per_second': 9.161, 'train_loss': 0.3042421261291442, 'epoch': 0.05}


TrainOutput(global_step=4909, training_loss=0.3042421261291442, metrics={'train_runtime': 535.8663, 'train_samples_per_second': 36.642, 'train_steps_per_second': 9.161, 'train_loss': 0.3042421261291442, 'epoch': 0.05})

In [21]:
print(metric.compute(predictions=fake_preds, references=fake_labels,))

{'accuracy': 0.5625}


In [22]:
#fitetuningpretrained pytorch from scratch
del model
del trainer
del args
#del batch
torch.cuda.empty_cache()